In [1]:
using ReverseDiff
using ReverseDiff: @forward
using ForwardDiff
using DiffResults

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/ReverseDiff.ji for module ReverseDiff.


In [67]:
module Nets

import Flux: relu, elu

abstract type Layer <: Function end

params(f::Function) = ()
with_params(f::Function) = f

struct Affine{TA <: AbstractMatrix, Tb <: AbstractVector} <: Layer
    A::TA
    b::Tb
end
    
params(a::Affine) = (a.A, a.b)
with_params(a::Affine, A, b) = Affine(A, b)

(a::Affine)(x) = a.A * x .+ a.b

struct Chain{L <: Tuple{Vararg{Function}}}
    layers::L
end

Chain(layers...) = Chain{typeof(layers)}(layers)

params(c::Chain) = _params(c.layers...)
_params(layer, layers...) = (params(layer)..., _params(layers...)...)
_params() = ()

(c::Chain)(x) = _chain_call(x, c.layers...)

_chain_call(x, layer, layers...) = _chain_call(layer(x), layers...)
_chain_call(x) = x

with_params(c::Chain, params...) = Chain(_with_params(c.layers, params)...)

function _with_params(layers, parameters)
    layer = first(layers)
    layer_params, remaining_params = _split(params(layer), parameters)
    (with_params(layer, layer_params...), _with_params(Base.tail(layers), remaining_params)...)
end

_with_params(::Tuple{}, ::Tuple{}) = ()


function _split(t1::NTuple{N1, Any}, t2::NTuple{N2, Any}) where {N1, N2}
    beginning, remainder = _split(Base.tail(t1), Base.tail(t2))
    (first(t2), beginning...), remainder
end

_split(::Tuple{}, t2::NTuple{N2, Any}) where {N2} = ((), t2)


end

INFO: Precompiling module Flux.


Nets

In [69]:
c = Nets.Chain(Nets.Affine(fill(1.0, 1, 1), fill(0.5, 1)),
    x -> @forward(Nets.elu).(x))

Nets.Chain{Tuple{Nets.Affine{Array{Float64,2},Array{Float64,1}},##7#8}}((Nets.Affine, #7))

In [70]:
Nets.params(c)

([1.0], [0.5])

In [71]:
c([1.0])

1-element Array{Float64,1}:
 1.5

In [72]:
params = (fill(1.0, 1, 1), [0.25])

([1.0], [0.25])

In [73]:
c2 = Nets.with_params(c, params...)

Nets.Chain{Tuple{Nets.Affine{Array{Float64,2},Array{Float64,1}},##7#8}}((Nets.Affine, #7))

In [74]:
c2([1.0])

1-element Array{Float64,1}:
 1.25

In [77]:
ReverseDiff.gradient((p...) -> Nets.with_params(c, p...)([1.0])[1], params)

([1.0], [1.0])